# Import Libraries

In [5]:
#Standard
import os
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np
import math as m
import pandas as pd
import scipy.stats as stats
from scipy.stats import iqr, kurtosis, skew
from tqdm import tnrange, tqdm_notebook
from statannot import add_stat_annotation

#import pillow (PIL) to allow for image cropping
import PIL 
from PIL import Image, ImageChops
from io import BytesIO

#image simplification and priming
#Convolution libraries
from scipy import signal
from skimage.measure import label, regionprops
from sklearn.preprocessing import Binarizer
#from sklearn.preprocessing import Binarizer
from scipy import ndimage


#Skimage used for direct detection ellipse
from skimage import io
from skimage import data, color, img_as_ubyte
from skimage.color import rgb2gray
from skimage.feature import canny
from skimage.transform import hough_ellipse
from skimage.draw import ellipse_perimeter
from skimage.transform import rescale, resize, downscale_local_mean

#Skimage used for direct detection circles
from skimage.transform import hough_circle, hough_circle_peaks
from skimage.feature import canny
from skimage.draw import circle_perimeter

#OpenCV for circle detection
import cv2

## define functions required

In [6]:
#This allows for the cutting of black space from each uCT image
def trim2(im,padding,offset):
    #selecting the outermost pixels 
    bg = Image.new(im.mode, im.size, im.getpixel((0,0)))
    diff = ImageChops.difference(im, bg)
    diff = ImageChops.add(diff, diff, 2.0, offset)
    bbox = diff.getbbox()
    #adding small boarder to each image 
    bbox = np.array(bbox).reshape(2,2)
    bbox[0] -= padding
    bbox[1] += padding
    bbox = bbox.flatten()
    bbox = tuple(bbox)
    if bbox:
        return bbox,padding

#obscure convolutes 2D arrays with a (x,y) sized screen screen and then binarizes them
def obscure(image_array,x,y,invert):
    screen = np.ones((x,y), dtype=int) 
    image_array = signal.convolve2d(image_array,screen, mode='same') #,mode='same')
    #convert image into binary
    #image_array = np.where(image_array > 127.5, 1, 0)
    if invert == 'yes':
        image_array = np.where(image_array > 127.5, 0, 1)
    elif invert == 'no':
        image_array = np.where(image_array > 127.5, 1, 0)
    return image_array

#largestblob can be used to detect the largest region of connected pixels and returns a filled image of those pixels and an approximate radius of that region
def largestblob(image_array,con):
    ##first label connected regions of pixels of an integer array
    #detect each areas with connected pixels 
    label_img, return_num = label(image_array,connectivity=con,return_num=True)
    if return_num >= 1:
        #calculate various statistics of each of the detected areas
        props = regionprops(label_img)
        #find the area in px of each of the detected areas
        areas = [r.area for r in props]
        #find the index of the largest areas
        indexOfMax = np.argmax(areas)
        #find the aproximate radius of the maximum area
        blob_radius =  round(props[indexOfMax].major_axis_length/2)
        blob = props[indexOfMax].filled_image
        blob_loc = props[indexOfMax].centroid
    else:
        blob = 0
        blob_radius = 0
        blob_loc = (0,0)
    return (blob,blob_radius,blob_loc)

# #largestblob can be used to detect the largest region of connected pixels and returns a filled image of those pixels and an approximate radius of that region
# def largestblob(image_array,con):
#     ##first label connected regions of pixels of an integer array
#     #detect each areas with connected pixels 
#     label_img, return_num = label(image_array,connectivity=con,return_num=True)
#     if return_num >= 1:
#         #calculate various statistics of each of the detected areas
#         props = regionprops(label_img)
#         #find the area in px of each of the detected areas
#         areas = [r.area for r in props]
#         #find the index of the largest areas
#         indexOfMax = np.argmax(areas)
#         #find the aproximate radius of the maximum area
#         blob_radius =  round(props[indexOfMax].major_axis_length/2)
#         blob = props[indexOfMax].filled_image
#         blob_loc = props[indexOfMax].centroid
#     else:
#         blob = 0
#         blob_radius = 0
#         blob_loc = (0,0)
#     return (blob,blob_radius,blob_loc)
    
        
#This allows for the additon of a padding to numpy array - useful for adding boarders to images
#found: https://docs.scipy.org/doc/numpy/reference/generated/numpy.pad.html
def pad_with(vector, pad_width, iaxis, kwargs):
    pad_value = kwargs.get('padder', 10)
    vector[:pad_width[0]] = pad_value
    vector[-pad_width[1]:] = pad_value
    return vector

# Loading Image Data

In [7]:
# #Initialisation of data read
# #current location
# location = os.getcwd()


# #/Users/ristomartin/Dropbox/UniStuff/DPhil/Experimental/ES_PCL_PDO_270219/
# #/Volumes/Seagate Backup Plus Drive/uCT/Reconstructed

# #Python folder location
# # loc = '/Users/ristomartin/dropbox/UniStuff/DPhil/Experimental/ES_PCL_PDO_270219/MicroCT/'#'/Volumes/Risto\'s SSD/uCT/Reconstructed/'
# #loc = '/Users/ristomartin/OneDrive/Dropbox/UniStuff/DPhil/Experimental/python_analysis/uCT/hollow_fibre'


# # loc = r'F:\Alex_Witt\uCT_Speed_Test\Rec_Files'
# #location for saved data
# save_loc = '/Users/ristomartin/OneDrive/Dropbox/UniStuff/DPhil/Experimental/python_analysis/uCT/hollow_fibre/output/'
# # save_loc = r'C:\Users\Alex Witt\Documents\Python_Analysis\Outputs'

# #where is data in location
# data_set = 'S4_10PPM_03_5PX_1_Rec'
# data_loc = loc+'/'+data_set
# #what to name to save files
# #savename = '686cl_9do_5ppm_060619_Rec'
# savename = data_set
# # #key location
# # key_loc = '/Users/ristomartin/Dropbox/UniStuff/DPhil/Experimental/ES_PCL_PDO_270219/sample_keys/'
# # #key to use
# # sample_key = key_loc + 'sample_key_130819.txt'
# # #location of raw_porosity distributions
# # raw_pore = '/Users/ristomartin/Dropbox/UniStuff/DPhil/Experimental/ES_PCL_PDO_270219/MicroCT/porosity_data/'

# # #Import filename key - insert key location
# # sample_key = pd.read_csv(sample_key,sep='\t')

# #initilisation of constants
# #Set conversion of px to um
# pxum = 5

# Processing Images

In [73]:
#Define Image processing script as function to be entered into multiprocessing
def fibrefeature(dat_loc,filename,pxum,fibre_pad,fibre_scale,fibre1,wire_diameter,lumen_pad,trim_offset,reverse,lumen1,lumen2,lumen3,debug,debug_print,save_pic):
    #check whether to save picture or not
    save_pic = save_pic
    #Intially open full image
    im = Image.open(dat_loc+'/'+filename)
    #check if file needs converting
    if im.mode == 'I;16':
        #specifying its sampling mode
        im.mode = 'I'
        #convert the mode into 'L' (8-bit pixels, black and white) and save as temporary file
        im = im.point(lambda i:i*(1./256)).convert('L')
        #open temporaray file
        #im = Image.open('temp.jpeg')
    elif im.mode == 'RGB' or im.mode == 'RGBA':
        im = im.convert('L')
    else:
        pass

    
    #Once image is opened make copies of unedited image and array to use later on
    #make copy of original unadultorated image
    im_orig = im.copy()
    #as well as an array of the unadultorated image
    im_orig_array = np.array(im_orig)
    #A = (A * B)
    #im_orig_array = im_orig_array*(255.0/im_orig_array.max())
    
    im_orig_array = im_orig_array.astype("uint8")
    
    #create plot of convoluted and binarised image
    if debug == True:
        fig, ax = plt.subplots()
        ax.imshow(im_orig_array, 'gray')
        if save_pic == True:
            ax.figure.savefig(save_loc+filename+'raw_image.png', dpi=300)
            
    ##Make selection of top RHS to find average pixel value to subtract
    bg_x1 = round((0.80*im_orig_array.shape[1]))
    bg_x2 = round((0.95*im_orig_array.shape[1]))
    bg_y1 = round((0.5*im_orig_array.shape[0]))
    bg_y2 = round((0.55*im_orig_array.shape[0]))
    
    bg_select = im_orig_array[bg_x1:bg_x2,bg_y1:bg_y2]
    
    # bg_select_df = pd.DataFrame(bg_select)
    # bg_select_df.to_csv(save_loc+savename+'bg_select.csv')

#     ret4,bg_select = cv2.threshold(im_orig_array,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    
    #create plot selected image background
    if debug == True:
        fig, ax = plt.subplots()
        ax.imshow(bg_select, 'gray')
        if save_pic == True:
            ax.figure.savefig(save_loc+filename+'_bg_select.png', dpi=300)
    
    height, width = bg_select.shape
    bg_med = []
    for i in range(0, width):
        #temp = (bg_select[i,:] > 0.1) * bg_select[i,:]
        temp = bg_select[i,:][bg_select[i,:]!=0]
        bg_med.extend(temp)

    #print(bg_med)
    bg_select_med = np.median(bg_med)+2*np.std(bg_med)  
    bg_select_mean = np.mean(bg_med)+2*np.std(bg_med)
    
    im_orig_array_c = im_orig_array.copy()
    
    #ret,bg_select = cv2.threshold(im_orig_array,bg_select_mean,im_orig_array.max(),cv2.THRESH_BINARY)
    im_orig_array_c[im_orig_array_c < bg_select_mean] = 0
    
    #create plot selected image background
    if debug == True:
        fig, ax = plt.subplots()
        ax.imshow(im_orig_array, 'gray')
        if save_pic == True:
            ax.figure.savefig(save_loc+filename+'_thresh_bg_select.png', dpi=300)
            
    im = Image.fromarray(im_orig_array_c)
    
    
    im_debug = np.array(im)
    #create plot selected image background
    if debug == True:
        fig, ax = plt.subplots()
        ax.imshow(im_debug, 'gray')
        if save_pic == True:
            ax.figure.savefig(save_loc+filename+'im_debug.png', dpi=300)

    #trim image to just pixels of interest using trim as defined above
    fibre_box,fpadding = trim2(im,(fibre_pad*2),trim_offset)
    im = im.crop(fibre_box)
    #convert trimmed image into numpy array
    nim =  np.array(im)
    #make copy of trimmed image to be used later on if needed
    nim_copy = nim.copy()
    
    #create plot of convoluted and binarised image
    if debug == True:
        fig, ax = plt.subplots()
        ax.imshow(nim_copy, 'gray')
        if save_pic == True:
            ax.figure.savefig(save_loc+filename+'cropped_image.png', dpi=300)
            
############################################################################################################################################################ 
                                                ### OUTER WALL DETECTION  ###
############################################################################################################################################################
    
    ##-- priming image for further analysis with obscure as defined above
    x = 7
    y = 7
    #Initially applying GayssuanBlur to minimise noise in image
    nim = cv2.GaussianBlur(nim,(x,y),0)
    #apply OTSU's binarisation method to strip away as much noise as possible and convert image into binary
    ret,fibre_thresh = cv2.threshold(nim,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    
    x = 2
    y = 2
    #1. Mark all pixels such that there are at least 6 pixels in their 7x7 neighborhood this will become defined as obscure function
    nim = signal.convolve2d(fibre_thresh,np.ones((x,y), dtype=int), mode='same')
    
    #Due to the porous nature of the hollow fibre membranes want to the dilate each of the detected regions into unified point
#     x = 7
#     y = 7
#     kernel = np.ones((x,y), np.uint8)

    #create plot of convoluted and binarised image
    if debug == True:
        fig, ax = plt.subplots()
        ax.imshow(nim, 'gray')
        if save_pic == True:
            ax.figure.savefig(save_loc+filename+'convolve2d_image.png', dpi=300)

    change = int(fibre1*max(nim.shape))
    if 1 > change:
        change = 1
    kernel = np.ones((change,change),np.uint8)
    
    nim = np.uint8(nim * 255)
    nim = cv2.dilate(nim, kernel, iterations=1)
    if trim_offset < -100:
        nim = ndimage.binary_erosion(np.uint8(nim), structure=kernel,iterations =3).astype(nim.dtype)
    else:
        nim = ndimage.binary_erosion(np.uint8(nim), structure=kernel,iterations =2).astype(nim.dtype)
    nim = ndimage.binary_dilation(nim,structure=kernel,iterations = 2).astype(nim.dtype)
    

    
    nim = ndimage.binary_dilation(nim,structure=kernel,iterations = 1).astype(nim.dtype)
    nim = ndimage.binary_erosion(np.uint8(nim), structure=kernel,iterations = 1).astype(nim.dtype)

    
    nim = np.uint8(nim * 255)
    
    #create plot of convoluted and binarised image
    if debug == True:
        fig, ax = plt.subplots()
        ax.imshow(nim, 'gray')
        if save_pic == True:
            ax.figure.savefig(save_loc+filename+'pre_processed_image.png', dpi=300)

        
    #2. Remove all blobs, but the largest
    dnim = rescale(nim, fibre_scale, anti_aliasing=False)
    dnim = np.uint8(dnim * 255)

    if debug == True:
        #create plot of filled image of largest region of connected pixels within convoluted and binarised image
        fig, ax = plt.subplots()
        ax.imshow(dnim, 'gray')
        if save_pic == True:
            ax.figure.savefig(save_loc+filename+'resized_pre_processed_image.png', dpi=300)
    
    
    #find largest blob in image
    bww,fibre_radius,fibre_loc = largestblob(dnim,1)
    
    #rescale fibre radius and location from resizing
    fibre_radius = fibre_radius/fibre_scale
    fibre_loc = ((fibre_loc[0]/fibre_scale),(fibre_loc[1]/fibre_scale))
    
    if debug == True:
        #create plot of filled image of largest region of connected pixels within convoluted and binarised image
        fig, ax = plt.subplots()
        ax.imshow(bww, 'gray')
        if save_pic == True:
            ax.figure.savefig(save_loc+filename+'first_largest_blob.png', dpi=300)
            
    #Padding
    bww = np.pad(bww, [(fibre_pad, fibre_pad), (fibre_pad, fibre_pad)], mode='constant')
    #nim_copy = np.pad(nim_copy, [(fibre_pad, fibre_pad), (fibre_pad, fibre_pad)], mode='constant')
    
    change = int(0.03*max(dnim.shape))
    if 1 > change:
        change = 1
    kernel = np.ones((change,change),np.uint8)
    
    bww = ndimage.binary_dilation(bww,structure=kernel,iterations = 2).astype(bww.dtype)
    
    if debug == True:
        #create plot of filled image of largest region of connected pixels within convoluted and binarised image
        fig, ax = plt.subplots()
        ax.imshow(bww, 'gray')
        if save_pic == True:
            ax.figure.savefig(save_loc+filename+'dilated_first_largest_blob.png', dpi=300)
    
    #bww = ndimage.binary_dilation(bww,structure=kernel,iterations = 1).astype(bww.dtype)
    bww = ndimage.binary_erosion(np.uint8(bww), structure=kernel,iterations = 2).astype(dnim.dtype)
    #bww = ndimage.binary_dilation(bww,structure=kernel,iterations = 1).astype(bww.dtype)
    
    if debug == True:
        #create plot of filled image of largest region of connected pixels within convoluted and binarised image
        fig, ax = plt.subplots()
        ax.imshow(bww, 'gray')
        if save_pic == True:
            ax.figure.savefig(save_loc+filename+'erroded_first_largest_blob.png', dpi=300)
    


    bww = np.uint8(bww*255)
       
    bww = largestblob(bww,2)[0]
    bww = np.pad(bww, [(fibre_pad, fibre_pad), (fibre_pad, fibre_pad)], mode='constant')

    if debug == True:
        #create plot of filled image of largest region of connected pixels within convoluted and binarised image
        fig, ax = plt.subplots()
        ax.imshow(bww, 'gray')
        if save_pic == True:
            ax.figure.savefig(save_loc+filename+'extracted_processed_first_largest_blob.png', dpi=300)
    

    #rescale image
    bww = rescale(bww, 1/fibre_scale, anti_aliasing=False)

    
    
    #add boarder to numpy array of both the largest blob of connected pixels detected as well as the image used to detect them.
    #want to add boarder as this allows for circles larger than the image to be drawn
    if fibre_radius > 0:
        #bww = np.pad(bww, [(fibre_pad, fibre_pad), (fibre_pad, fibre_pad)], mode='constant')
        
        if debug == True:
            #create plot of filled image of largest region of connected pixels within convoluted and binarised image
            fig, ax = plt.subplots()
            ax.imshow(bww, 'gray')
            if save_pic == True:
                ax.figure.savefig(save_loc+filename+'largest_blob.png', dpi=300)      


        #3. Fill holes
        #- this is not done as is auto in python

        #4. Apply edge detection
        #find edges in data
        edges = canny(bww, sigma=3)
        edges = np.uint8(edges * 255)
        
        if debug == True:
            #create plot of filled image of largest region of connected pixels within convoluted and binarised image
            fig, ax = plt.subplots()
            ax.imshow(edges, 'gray')
            if save_pic == True:
                ax.figure.savefig(save_loc+filename+'largest_blob_edges.png', dpi=300)   


        #5. fit ellipse onto total fibre with cv2.fitellipse
        #detect contours from edges
        #for explination of hiarichy choice 'cv2.RETR_EXTERNAL' check:https://docs.opencv.org/3.1.0/d9/d8b/tutorial_py_contours_hierarchy.html
        #for explination of approximation choice 'cv2.CHAIN_APPROX_SIMPLE' check: https://docs.opencv.org/3.1.0/d3/dc0/group__imgproc__shape.html#ga4303f45752694956374734a03c54d5ff
        contours = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2]
        #Select the external contours which collude to give the largest area to ensure outer points are taken
        contours = max(contours, key=cv2.contourArea)
        #print(contours)

        #fit ellipse to contour points
        #https://docs.opencv.org/3.0-beta/doc/py_tutorials/py_imgproc/py_contours/py_contour_properties/py_contour_properties.html
        #paper published on method used to find elipse https://pdfs.semanticscholar.org/19d9/becce00a500bdd1cad5a19f9f16175347096.pdf
        (fibre_x,fibre_y),(FMA,fma),fangle = cv2.fitEllipse(contours)
        FMA = int(FMA)
        fma = int(fma)
        fibre_x = int(fibre_x)
        fibre_y = int(fibre_y)
        faspect_ratio = FMA/fma

        #first get the shape of the trimmed image
        H, W = nim_copy.shape
        image_diagonal = m.sqrt((H**2)+(W**2))

        #put a check into the  ensure the detected ellipse is not larger than the image its self as this is a false detection            
        if FMA < image_diagonal and fma < image_diagonal:
            #draw ellipse onto edges
            #having found the circle then plot the circle onto the image
            image = edges
            #convert the image into rgb to allow for plotting of red circle
            image = color.gray2rgb(image)
            #simiarly do this to the array of the array of the trimmed image
            nim_copy_draw = color.gray2rgb(nim_copy)
            
            if debug == True:
                #create plot of filled image of largest region of connected pixels within convoluted and binarised image
                fig, ax = plt.subplots()
                ax.imshow(nim_copy_draw, 'gray')
                if save_pic == True:
                    ax.figure.savefig(save_loc+filename+'nim_copy_draw.png', dpi=300)   



            cv2.ellipse(image,((fibre_x,fibre_y),(FMA,fma),fangle),(0,0,255),2)
            
            cv2.ellipse(nim_copy_draw,((fibre_loc[1],fibre_loc[0]),(FMA,fma),fangle),(0,0,255),2)
            
            
############################################################################################################################################################ 
                        ###IF NO ELLIPSE DETECTED TRY DETECT HOUGH CIRCLE###
############################################################################################################################################################ 
            ellipse_fail = False
        else:
            ellipse_fail = True
            if ellipse_fail==True:
                pass
            else:
                print('hough')
                #If incorrectly identify ellipse, try and detect with circle instead
                #4.5 fit circle to detected edges 
                #As before can use the aproximate lumen radius to aproximate the size of the lumen
                test_radii = np.flip(np.arange((fibre_radius-20), (fibre_radius+20), 1))
                #print(test_radii)

                #Using the approximate lumen size can then feed these into Hough filter to find regions of greatest interaction
                hough_res = hough_circle(edges, test_radii)

                # Select the most prominent circle
                accums, fibre_x, fibre_y, FMA = hough_circle_peaks(hough_res,test_radii, min_xdistance=5, min_ydistance=5,  total_num_peaks=1)
                #have the centrioid of the largest block detected within the image which is presumably and the centre of the blob image in the larger image
                #fibre_x and fibre_y are the centroid of the ellipse or circle detected in the smaller image by finding the distance from the smaller image
                #centroid can compute the adjustment add onto the larger image blob centroid to correctly position ellipse in larger image
                foffset_x = (bww.shape[1]/2)-fibre_loc[1]
                foffset_y = (bww.shape[0]/2)-fibre_loc[0]
                #correcting 
                fibre_x = (fibre_x - foffset_x).astype(int)
                fibre_y = (fibre_y - foffset_y).astype(int)
                #print(type(FMA))
                FMA = FMA.astype(int)

                #having found the circle then plot the circle onto the image convert the image into rgb to allow for plotting of red circle
                image = color.gray2rgb(edges)
                #simiarly do this to the array of the array of the trimmed image
                #print(type(nim_copy))
                nim_copy_draw = color.gray2rgb(nim_copy)


                #draw the circle from the hough circle detection onto both the edges diagram and trimmed image
                for center_y, center_x, radius in zip(fibre_y, fibre_x, FMA):
                    circy, circx = circle_perimeter(center_y, center_x, radius)
                    image[circy, circx] = (220, 20, 20)
                    nim_copy_draw[circy, circx] = (220, 20, 20)

                if debug == True:
                    #create plot of filled image of largest region of connected pixels within convoluted and binarised image
                    fig, ax = plt.subplots()
                    ax.imshow(image, 'gray')
                    if save_pic == True:
                        ax.figure.savefig(save_loc+filename+'circle_on_edges.png', dpi=300)
                    #create plot of filled image of largest region of connected pixels within convoluted and binarised image
                    fig, ax = plt.subplots()
                    ax.imshow(nim_copy_draw, 'gray')
                    if save_pic == True:
                        ax.figure.savefig(save_loc+filename+'circle_on_image.png', dpi=300) 


                #make the minor fibre axis equal to that as major axis, as this is true of a circle
                fma = FMA
############################################################################################################################################################ 
                                                ### ISOLATION OF LUMEN ROI  ###
############################################################################################################################################################


        #check that if that the circle detected is not larger or less than 1/5th the size of the total image
        if FMA < image_diagonal and (image_diagonal/10) < FMA:
            if ellipse_fail == True:
                pass
            else:

                ##-Having detected either a circle of ellipse use centre points to detect ROI
                #Want to crop out only the region of interest
                #To find the lumen of the fibre want can consider the central region of the detected fibre only as fibre will form around the wire
                #set the size of the area of interest - as we are using a wire of 100um and the px size is ~1um use box width slightly larger to capture lumen
                square_width = wire_diameter/pxum
                #fibre_centre_x = center_x
                #fibre_centre_y = center_y
                x1 = int((fibre_loc[1])-(square_width/2))
                y1 = int((fibre_loc[0])-(square_width/2))
                x2 = int((fibre_loc[1])+(square_width/2))
                y2 =int((fibre_loc[0])+(square_width/2))
                square_nwc  = (x1,y1)
                square_sec =(x2,y2)
                
                if debug_print == True:
                    print('x1 ='+str(x1))
                    print('y1 ='+str(y1))
                    print('x2 ='+str(x2))
                    print('y2 ='+str(y2))
                
                
                #having defined the corners of the central region of the fibre can draw circle
                #cv2.rectangle(image,(fibre_x-(square_width/2),fibre_y-(square_width/2)),(fibre_x+(square_width/2),fibre_y+(square_width/2)),(0,255,0),3)
                cv2.rectangle(nim_copy_draw,(x1,y1),(x2,y2),(0,255,0),3)
                
                

                if debug == True:
                    #create plot of filled image of largest region of connected pixels within convoluted and binarised image
                    fig, ax = plt.subplots()
                    ax.imshow(image, 'gray')
                    if save_pic == True:
                        ax.figure.savefig(save_loc+filename+'ellipse_on_edges.png', dpi=300)
                    #create plot of filled image of largest region of connected pixels within convoluted and binarised image
                    fig, ax = plt.subplots()
                    ax.imshow(nim_copy_draw, 'gray')
                    if save_pic == True:
                        ax.figure.savefig(save_loc+filename+'ellipse_on_image.png', dpi=300) 



                #Extracting ROI from image slice array
                #having identified the outer circle and the central region within the image (where we should find the lumen) want to extract the region of interest
                roi = nim_copy[y1:y2, x1:x2]

                #check the shape of the roi extracted - if either is found to be 0 centre of detected region is off image and therefore an invalid fit
                roi_H, roi_W = roi.shape
                if roi_H != 0 or roi_W !=0:
                    if debug == True:
                        #create plot of filled image of largest region of connected pixels within convoluted and binarised image
                        fig, ax = plt.subplots()
                        ax.imshow(nim_copy_draw, 'gray')
                        if save_pic == True:
                            ax.figure.savefig(save_loc+filename+'image_with_roi_box.png', dpi=300)
                        #create plot of filled image of largest region of connected pixels within convoluted and binarised image
                        fig, ax = plt.subplots()
                        ax.imshow(roi, 'gray')
                        if save_pic == True:
                            ax.figure.savefig(save_loc+filename+'raw_ROI.png', dpi=300)


    ############################################################################################################################################################ 
                                                    ### LUMEN DETECTION  ###
    ############################################################################################################################################################

                    #having extracted roi for lumen - want to peform similar transform as for entire image
                    image_array = roi
                    #apply thresholding using value calculated for total image before with OTSU's with inversed image as want to detect negitive space
                    # ret2,image_array = cv2.threshold(image_array,ret,255,cv2.THRESH_BINARY_INV)
                    
                    #image_array, disc_array = cv2.threshold(roi,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
                    
                    ret2, image_array = cv2.threshold(image_array,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
                    
                    

                    if debug == True:
                        #create plot of filled image of largest region of connected pixels within convoluted and binarised image
                        fig, ax = plt.subplots()
                        ax.imshow(image_array, 'gray')
                        if save_pic == True:
                            ax.figure.savefig(save_loc+filename+'pre_processing_lumen_image.png', dpi=300)
                            
                    change = int(lumen1*pxum*max(image_array.shape))
                    if 1 > change:
                        change = 1
                    kernel = np.ones((change,change),np.uint8)
                    
                    if reverse == True:
                        image_array = ndimage.binary_erosion(image_array, structure=kernel,iterations =4).astype(image_array.dtype)
                    else:
                        image_array = ndimage.binary_dilation(image_array,structure=kernel,iterations =4).astype(image_array.dtype)
                    
                    
                    
                    if debug == True:
                        #create plot of filled image of largest region of connected pixels within convoluted and binarised image
                        fig, ax = plt.subplots()
                        ax.imshow(image_array, 'gray')
                        if save_pic == True:
                            ax.figure.savefig(save_loc+filename+'pre-erroded_lumen_image.png', dpi=300)


                    x = 2
                    y = 2
                    #1. Mark all pixels such that there are at least 6 pixels in their 7x7 neighborhood this will become defined as obscure function
                    #image_array = signal.convolve2d(image_array,np.ones((x,y), dtype=int), mode='same')
                    image_array = np.uint8(image_array)
                    
                    ret2, image_array = cv2.threshold(image_array,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
                    
                    if debug == True:
                        #create plot of filled image of largest region of connected pixels within convoluted and binarised image
                        fig, ax = plt.subplots()
                        ax.imshow(image_array, 'gray')
                        if save_pic == True:
                            ax.figure.savefig(save_loc+filename+'convolved_lumen_image.png', dpi=300)
                    #0.02
                    change = int(lumen2*pxum**max(image_array.shape))
                    if 1 > change:
                        change = 1
                    kernel = np.ones((change,change),np.uint8)
                    
                    #image_array = cv2.dilate(image_array, kernel, iterations=1)
                    if reverse == True:
                        image_array = ndimage.binary_erosion(image_array, structure=kernel,iterations = 2).astype(image_array.dtype)
                    else:
                        image_array = ndimage.binary_dilation(image_array,structure=kernel,iterations = 2).astype(image_array.dtype)
                    
                    if debug == True:
                        #create plot of filled image of largest region of connected pixels within convoluted and binarised image
                        fig, ax = plt.subplots()
                        ax.imshow(image_array, 'gray')
                        if save_pic == True:
                            ax.figure.savefig(save_loc+filename+'initial_lumen_dilate.png', dpi=300)

                    #0.015
                    change = int(lumen3*pxum**max(image_array.shape))
                    if 1 > change:
                        change = 1
                    kernel = np.ones((change,change),np.uint8)
                    
                    image_array = ndimage.binary_erosion(image_array, structure=kernel,iterations =2).astype(image_array.dtype)
                    image_array = ndimage.binary_dilation(image_array,structure=kernel,iterations = 2).astype(image_array.dtype)
                    
                    
#                     image_array = ndimage.binary_erosion(np.uint8(image_array), structure=kernel).astype(dnim.dtype)

#                     image_array = cv2.dilate(image_array, kernel, iterations=1)

                    image_array = cv2.threshold(image_array,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]
                    #image_array = np.uint8(image_array * 255)

                    #create plot of convoluted and binarised image
                    if debug == True:
                        fig, ax = plt.subplots()
                        ax.imshow(image_array, 'gray')
                        if save_pic == True:
                            ax.figure.savefig(save_loc+filename+'processed_lumen_image.png', dpi=300)


                    # #2. Remove all blobs, but the largest
                    # #downsize image
                    # scale = 0.5
                    # image_array = rescale(image_array, scale, anti_aliasing=False)
                    # image_array = np.uint8(image_array * 255)

                    # if debug == True:
                    #     #create plot of filled image of largest region of connected pixels within convoluted and binarised image
                    #     fig, ax = plt.subplots()
                    #     ax.imshow(image_array, 'gray')
                    #     if save_pic == True:
                    #         ax.figure.savefig(save_loc+filename+'resized_pre_processed_lumen_image.png', dpi=300)


                    #find largest blob in image
                    lumen_blob,aprox_lumen_radius,lumen_blob_loc = largestblob(image_array,2)

                    # #rescale fibre radius and location from resizing
                    # lumen_radius = aprox_lumen_radius/scale
                    # lumen_blob_loc = ((lumen_blob_loc[0]/scale),(lumen_blob_loc[1]/scale))

                    if debug == True:
                        #create plot of filled image of largest region of connected pixels within convoluted and binarised image
                        fig, ax = plt.subplots()
                        ax.imshow(image_array, 'gray')
                        if save_pic == True:
                            ax.figure.savefig(save_loc+filename+'first_largest_blob_lumen.png', dpi=300)                

                    #Padding
                    lumen_blob = np.pad(lumen_blob, [(fibre_pad, fibre_pad), (fibre_pad, fibre_pad)], mode='constant')

                    #having padded the image take note of new size to allow for localisation of circles later on
                    lumen_shape = lumen_blob.shape


                    if debug == True:
                        #create plot of filled image of largest region of connected pixels within convoluted and binarised image
                        fig, ax = plt.subplots()
                        ax.imshow(lumen_blob, 'gray')
                        if save_pic == True:
                            ax.figure.savefig(save_loc+filename+'ROI_largest_blob.png', dpi=300)


                    #4. Apply edge detection
                    #find edges in data
                    lumen_edges = np.uint8(canny(lumen_blob, sigma=0.1)* 255)

                    if debug == True:
                        #create plot of filled image of largest region of connected pixels within convoluted and binarised image
                        fig, ax = plt.subplots()
                        ax.imshow(lumen_edges, 'gray')
                        if save_pic == True:
                            ax.figure.savefig(save_loc+filename+'ROI_edges.png', dpi=300)


                    #5. Find circle using with cv2.fitellipse
                    contours = cv2.findContours(lumen_edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2]
                    #Select the external contours which collude to give the largest area to ensure outer points are taken
                    contours = max(contours, key=cv2.contourArea)

                    #fit ellipse to contour points
                    #https://docs.opencv.org/3.0-beta/doc/py_tutorials/py_imgproc/py_contours/py_contour_properties/py_contour_properties.html
                    #paper published on method used to find elipse https://pdfs.semanticscholar.org/19d9/becce00a500bdd1cad5a19f9f16175347096.pdf
                    (elumen_x,elumen_y),(LMA,lma),langle = cv2.fitEllipse(contours)
                    LMA = int(LMA)
                    lma = int(lma)
                    elumen_x = int(elumen_x)
                    elumen_y = int(elumen_y)
                    laspect_ratio = LMA/lma

                    #letting the detected edges of the lumen be background can now plot these as a check
                    image = lumen_edges
                    image = color.gray2rgb(image)

                    #fit ellipse to contour points
                    #https://docs.opencv.org/3.0-beta/doc/py_tutorials/py_imgproc/py_contours/py_contour_properties/py_contour_properties.html
                    #paper published on method used to find elipse https://pdfs.semanticscholar.org/19d9/becce00a500bdd1cad5a19f9f16175347096.pdf
                    (elumen_x,elumen_y),(LMA,lma),langle = cv2.fitEllipse(contours)
                    LMA = int(LMA)
                    lma = int(lma)
                    elumen_x = int(elumen_x)
                    elumen_y = int(elumen_y)
                    laspect_ratio = LMA/lma

                    #letting the detected edges of the lumen be background can now plot these as a check
                    image = lumen_edges
                    image = color.gray2rgb(image)

                    #draw ellipse onto edges
                    cv2.ellipse(image,((elumen_x,elumen_y),(LMA,lma),langle),(0,0,255),2)

                    if debug == True:
                        #create plot of filled image of largest region of connected pixels within convoluted and binarised image
                        fig, ax = plt.subplots()
                        ax.imshow(image, 'gray')
                        if save_pic == True:
                            ax.figure.savefig(save_loc+filename+'ROI_ellipse.png', dpi=300)

                    #Similarly transform centre of ROI detected by cv2.fitellipse
                    #having found the parimeter of the lumen now want to plot back in orignial image


                    lumen_plot_diffy = square_width - np.shape(lumen_edges)[0]
                    lumen_plot_diffx = square_width - np.shape(lumen_edges)[1]

                    #adjusting this for the larger image - add on the points from the origin of the roi within the larger image aka top left
                    # print('square_width ='+str(square_width))
                    # print('y2 ='+str(y2))
                    # print('elumen_y ='+str(elumen_y))
                    # print('lumen_plot_diffy ='+str(lumen_plot_diffy))
                    # print('lumen_plot_diffx ='+str(lumen_plot_diffx))
                    eroi_x1 = lumen_blob_loc[1]+x1
                    eroi_y1 = lumen_blob_loc[0]+y1


                    #cv2.ellipse(nim_copy_draw,((fibre_loc[1]+fibre_pad,fibre_loc[0]+fibre_pad),(FMA,fma),fangle),(0,0,255),2)

                    #draw  the ellipse outline of the lumen onto the original image

                    cv2.ellipse(nim_copy_draw,((lumen_blob_loc[1]+x1,lumen_blob_loc[0]+y1),(LMA,lma),langle),(0,0,255),2)
                    #cv2.ellipse(nim_copy_draw,((eroi_x1,eroi_y1),(LMA,lma),langle),(0,0,255),2)

                    if debug == True:
                        #create plot of filled image of largest region of connected pixels within convoluted and binarised image
                        fig, ax = plt.subplots()
                        ax.imshow(nim_copy_draw, 'gray')
                        if save_pic == True:
                            ax.figure.savefig(save_loc+filename+'cropped_image_w_ROI.png', dpi=300)

############################################################################################################################################################ 
                                        ### Determining porosity  ###
############################################################################################################################################################



                    fibre_mask = np.zeros((nim_copy.shape[0],nim_copy.shape[1]))
                    #print(FMA,fma)
                    cv2.ellipse(fibre_mask,((((fibre_loc[1])),(fibre_loc[0])),(FMA,fma),fangle),255, -1)
                    cv2.ellipse(fibre_mask,((eroi_x1,eroi_y1),(LMA,lma),langle),0,-1)

                    #test to see if mask looks correct
                    if debug == True:
                        fig15, ax15 = plt.subplots()
                        ax15.imshow(fibre_mask, cmap=plt.cm.gray)
                        if save_pic == True:
                            ax15.figure.savefig(save_loc+filename+'fibrepore_15.png', dpi=300)

                    #creating masked image
                    masked_image = np.where(fibre_mask == 255, nim_copy, 0)

                    #test to see if mask is applied correctly
                    if debug == True:
                        fig16, ax16 = plt.subplots()
                        ax16.imshow(masked_image, cmap=plt.cm.gray)
                        if save_pic == True:
                            ax16.figure.savefig(save_loc+filename+'fibrepore_16.png', dpi=300)


                    # having produced a masked image of the region of interest want to binarize
                    # Neither the adaptive thresholding methods appropriate as act more as edge detection rather than global binarization
                    #to overcome the arbatory nature of global threshold use Otsu's method which detects the separtation in the pixel hiistogram to define cut off point
                    #OTSU Threshold

                    ret4,th5 = cv2.threshold(masked_image,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
                    if debug == True:
                        fig17, ax17 = plt.subplots()
                        ax17.imshow(th5,'gray')
                        if save_pic == True:
                            ax17.figure.savefig(save_loc+filename+'fibrepore_17.png', dpi=300)
                    try:
                        #Having converted into binary then look at ratio of pixels between white and black to get porosity
                        mask_white_pxls = fibre_mask.sum()
                        fibre_white_pxls = th5.sum()
                        fibre_black_pxls = mask_white_pxls - fibre_white_pxls
                        porosity = (fibre_black_pxls/mask_white_pxls)*100
                        # print(porosity)

############################################################################################################################################################ 
                                        ### Determining wall thickness  ###
############################################################################################################################################################

                        a = np.linspace(0,2*np.pi,num=100)
                        rfangle = fangle * (np.pi/180)
                        rfma = fma/2
                        rFMA = FMA/2

                        # fx = (rfma*np.cos(a)*np.cos(rfangle))-(rFMA*np.sin(a)*np.sin(rfangle))
                        # fy = (rfma*np.cos(a)*np.sin(rfangle))+(rFMA*np.sin(a)*np.cos(rfangle))
                        fx = (rFMA*np.cos(a)*np.cos(rfangle))-(rfma*np.sin(a)*np.sin(rfangle))
                        fy = (rFMA*np.cos(a)*np.sin(rfangle))+(rfma*np.sin(a)*np.cos(rfangle))
                        
                        fr = [np.sqrt((fx[i]**2)+(fy[i]**2)) for i in range(len(fx))]# if fy[i]>0]
                        f0 = [m.atan2(fy[i],fx[i]) for i in range(len(fx))]# if fy[i]>0]

                        fd_df = pd.DataFrame()
                        fd_df['fr'] = fr
                        fd_df['f0'] = f0
                        fd_df = fd_df.sort_values(by=['f0'], ascending=False)
                        # print(fd_df.head())

                        fr = fd_df['fr'].to_list()
                        f0 = fd_df['f0'].to_list()

                       # print(fr)
                        #print(f0)
                        # print('FMA='+str(FMA))
                        # print('fma='+str(fma))

                        rlangle = langle * (np.pi/180)
                        rlma = lma/2
                        rLMA = LMA/2
                        # lx = (rlma*np.cos(a)*np.cos(rlangle))-(rLMA*np.sin(a)*np.sin(rlangle))-(fibre_loc[1]-eroi_x1)
                        # ly = (rlma*np.cos(a)*np.sin(rlangle))+(rLMA*np.sin(a)*np.cos(rlangle))-(fibre_loc[0]-eroi_y1)
                        lx = (rLMA*np.cos(a)*np.cos(rlangle))-(rlma*np.sin(a)*np.sin(rlangle))-(fibre_loc[1]-eroi_x1)
                        ly = (rLMA*np.cos(a)*np.sin(rlangle))+(rlma*np.sin(a)*np.cos(rlangle))-(fibre_loc[0]-eroi_y1)

                        lr = [np.sqrt((lx[i]**2)+(ly[i]**2)) for i in range(len(lx))]# if ly[i]>0]
                        l0 = [m.atan2(ly[i],lx[i]) for i in range(len(lx))]# if ly[i]>0]

                        ld_df = pd.DataFrame()
                        ld_df['lr'] = lr
                        ld_df['l0'] = l0
                        #ld_df = ld_df.sort_values(by=['l0'], ascending=False)
                        # print(ld_df.head())

                        lr = ld_df['lr'].to_list()
                        l0 = ld_df['l0'].to_list()
                        #print(lr)
                        #print(l0)

                        if debug == True:
                            #plot polar coordinates
                            fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
                            ax.plot(l0, lr,'b')
                            ax.plot(f0, fr,'r')
                            if save_pic == True:
                                ax.figure.savefig(save_loc+filename+'polar_output.png', dpi=300)

                        # print(fr)
                        # print(f0)
                        # print(lr)
                        # print(l0)

                        distances = [np.sqrt(((fr[i]**2)+(lr[i]**2))-(2*fr[i]*lr[i]*np.cos(l0[i]-f0[i]))) for i in range(len(l0))]

                        # print(distances)




                        #find the minimum and maximum distance
                        max_dis = np.amax(np.asarray(distances))
                        min_dis = np.amin(np.asarray(distances))
                        #find average 
                        wall_median = np.percentile(np.asarray(distances), 50)*pxum
                        wall_mean = np.asarray(distances).mean()*pxum
                        wall_skew = skew(np.asarray(distances))
                        #from calculation of skew determine whether to use median or mean
                        if abs(wall_skew) > 0.5:
                            wall_stat = 'median'
                        else:
                            wall_stat = 'mean'

############################################################################################################################################################ 
                                        ### Save out  ###
############################################################################################################################################################

                        return (porosity,max_dis,min_dis,wall_median,wall_mean,wall_skew,wall_stat,faspect_ratio,laspect_ratio)

                    except:
                        pass
                else:
                    pass

        else:
            pass
    else:
        pass 

In [77]:
test = False
show_all = True
debug_print = False

columns = ['porosity','max_dis','min_dis','wall_median','wall_mean','wall_skew','wall_stat','faspect_ratio','laspect_ratio']

cfp = pd.DataFrame(columns = columns)

if test == True:
    
    #where is data located    
    #loc = '/Volumes/RISTO_EXHDD/Alex_Witt/uCT_Speed_Test/Rec_Files/'
    loc = '/Users/ristomartin/OneDrive/Dropbox/UniStuff/DPhil/Experimental/python_analysis/uCT/hollow_fibre'
    # loc = '/Volumes/Ristos_SSD/uCT/ROIs/'
    
    #What is the name of the data set?
    data_set = 'Test'#'S4_10PPM_03_5PX_1_Rec'
    
    data_loc = loc+'/'+data_set
    
    #location for saved data
    save_loc = '/Users/ristomartin/OneDrive/Dropbox/UniStuff/DPhil/Experimental/python_analysis/uCT/hollow_fibre/output/'
    # save_loc = r'C:\Users\Alex Witt\Documents\Python_Analysis\Outputs'
    
    #what to name to save files
    savename = data_set

    #initilisation of constants - Set conversion of px to um
    pxum = 1
    wire_diameter = 300
    fibre_pad = 50
    lumen_pad = 50
    trim_offset = -80
    
    #Generate list of file in data location
    files = [x for x in os.listdir(data_loc) if x.endswith(('.tif','.jpg','.png','.bmp'))==True and x.startswith('._')==False]
    
    #Itterating through files in data location
    for filename in tqdm_notebook(files):
        print(filename)
        
        #Lumen 1 - dilation
        #Lumen 2 - errosion
        #Lumen 3 - errosion then dilation
        #acertain fibre properties as defined above
        if show_all == True:
            fibre_properties = fibrefeature(data_loc,filename,pxum,fibre_pad,0.25,0.05,wire_diameter,lumen_pad,trim_offset,False,0.01,0.3,0.3,True,True,True)
        else:
            fibre_properties = fibrefeature(data_loc,filename,pxum,fibre_pad,0.25,wire_diameter,lumen_pad,trim_offset,False,0.02,0.2,0.3,False,False,False)
        if fibre_properties is None:
            pass
        else:
            # print(fibre_properties)
            print('porosity ='+str(fibre_properties[0]))
            print('max_dis ='+str(fibre_properties[1]))
            print('min_dis ='+str(fibre_properties[2]))
            print('wall_median ='+str(fibre_properties[3]))
            print('wall_mean ='+str(fibre_properties[4]))
            print('wall_skew ='+str(fibre_properties[5]))
            print('wall_stat ='+str(fibre_properties[6]))
            print('faspect_ratio ='+str(fibre_properties[7]))
            print('laspect_ratio ='+str(fibre_properties[8]))
            cfp = cfp.append({'filename':filename,'porosity':fibre_properties[0],'max_dis':fibre_properties[1],'min_dis':fibre_properties[2],'wall_median':fibre_properties[3],
                          'wall_mean':fibre_properties[4],'wall_skew':fibre_properties[5],'wall_stat':fibre_properties[6],'faspect_ratio':fibre_properties[7],
                          'laspect_ratio':fibre_properties[8]}, ignore_index=True)
    print(cfp.head())
    cfp.to_csv(save_loc+savename+'.csv')

        


In [79]:
Run = True

columns = ['porosity','max_dis','min_dis','wall_median','wall_mean','wall_skew','wall_stat','faspect_ratio','laspect_ratio']

cfp = pd.DataFrame(columns = columns)

if Run == True:
    
    #where is data located    
    #/Users/ristomartin/Dropbox/UniStuff/DPhil/Experimental/ES_PCL_PDO_270219/
    #/Volumes/Seagate Backup Plus Drive/uCT/Reconstructed
    #/Volumes/Risto's SSD/uCT/Reconstructed
    #Python folder location
    # loc = '/Users/ristomartin/dropbox/UniStuff/DPhil/Experimental/ES_PCL_PDO_270219/MicroCT/'#'/Volumes/Risto\'s SSD/uCT/Reconstructed/'
    #loc = '/Users/ristomartin/OneDrive/Dropbox/UniStuff/DPhil/Experimental/python_analysis/uCT/hollow_fibre'
    #loc = '/Volumes/RISTO_EXHDD/Alex_Witt/uCT_Speed_Test/Rec_Files/'
    loc = '/Volumes/Ristos_SSD/uCT/ROIs/'

    # loc = r'F:\Alex_Witt\uCT_Speed_Test\Rec_Files'
    #location for saved data
    save_loc = '/Users/ristomartin/OneDrive/Dropbox/UniStuff/DPhil/Experimental/python_analysis/uCT/hollow_fibre/output/'
    # save_loc = r'C:\Users\Alex Witt\Documents\Python_Analysis\Outputs'

    #where is data in location
    data_set = '305cl_4do_S1_5ppm_120619_Rec' #S4_10PPM_03_5PX_1_Rec
    data_loc = loc+data_set
    #what to name to save files
    savename = data_set

    #initilisation of constants - Set conversion of px to um
    pxum = 1
    wire_diameter = 300
    fibre_pad = 50
    lumen_pad = 50
    trim_offset = -80
    
    
    #Generate list of file in data location
    files = [x for x in os.listdir(data_loc) if x.endswith(('.tif','.jpg','.png','.bmp'))==True and x.startswith('._')==False]
    
    #Itterating through files in data location
    for filename in tqdm_notebook(files):
        #print filename to track any error
        print(filename)

        #acertain fibre properties as defined above
        fibre_properties = fibrefeature(data_loc,filename,pxum,fibre_pad,0.25,0.05,wire_diameter,lumen_pad,trim_offset,False,0.01,0.3,0.3,False,False,False)
        if fibre_properties is None:
            pass
        else:        
            cfp = cfp.append({'filename':filename,'porosity':fibre_properties[0],'max_dis':fibre_properties[1],'min_dis':fibre_properties[2],'wall_median':fibre_properties[3],
                          'wall_mean':fibre_properties[4],'wall_skew':fibre_properties[5],'wall_stat':fibre_properties[6],'faspect_ratio':fibre_properties[7],
                          'laspect_ratio':fibre_properties[8]}, ignore_index=True)
    print(cfp.head())
    cfp.to_csv(save_loc+savename+'.csv')


/var/folders/dw/7hyg5yn15331npl8_ss6cl_40000gn/T/ipykernel_9930/3928021706.py:42: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for filename in tqdm_notebook(files):


  0%|          | 0/2652 [00:00<?, ?it/s]

305cl_4do_5ppm_120619_rec_roi_0015.tif
305cl_4do_5ppm_120619_rec_roi_0016.tif
305cl_4do_5ppm_120619_rec_roi_0017.tif
305cl_4do_5ppm_120619_rec_roi_0018.tif
305cl_4do_5ppm_120619_rec_roi_0019.tif
305cl_4do_5ppm_120619_rec_roi_0020.tif
305cl_4do_5ppm_120619_rec_roi_0021.tif
305cl_4do_5ppm_120619_rec_roi_0022.tif
305cl_4do_5ppm_120619_rec_roi_0023.tif
305cl_4do_5ppm_120619_rec_roi_0024.tif
305cl_4do_5ppm_120619_rec_roi_0025.tif
305cl_4do_5ppm_120619_rec_roi_0026.tif
305cl_4do_5ppm_120619_rec_roi_0027.tif
305cl_4do_5ppm_120619_rec_roi_0028.tif
305cl_4do_5ppm_120619_rec_roi_0029.tif
305cl_4do_5ppm_120619_rec_roi_0030.tif


KeyboardInterrupt: 

# Processing Image Data

## Unification of data

In [ ]:
raw_pore = save_loc+'MicroCT/porosity_data/'
print(raw_pore)

In [ ]:
#Label each of the columns to be in final dataframe
#uni_columns = ['fmajor_axis', 'fminor_axis', 'faspect_ratio', 'lmajor_axis', 'lminor_axis', 'laspect_ratio', 'porosity','fwall_mthickness']

#create dataframe for all data
cdf = pd.DataFrame()

for filename in os.listdir(raw_pore+'preprocessed/'):
    #only open the file if it end with the specified parameters as specified in file name
    if filename.endswith(".csv"):    
        #reads the specified directory and opens it as a dataframe
        df = pd.read_csv(os.path.join(raw_pore+'preprocessed/'+filename),index_col=0)
        
        #print(df.head())
        
        #Copy process parameters from sample key
        df['Pyridine Concentration'] = sample_key.loc[sample_key['uCT_filename'] == filename, 'pyridine_conc'].iloc[0]
        df['Wire Speed'] = sample_key.loc[sample_key['uCT_filename'] == filename, 'wire_speed'].iloc[0]
        df['Rotation Speed'] = sample_key.loc[sample_key['uCT_filename'] == filename, 'rotation_speed'].iloc[0]
        df['Polymer Solution'] = sample_key.loc[sample_key['uCT_filename'] == filename, 'solution_name'].iloc[0]
        voltage = sample_key.loc[sample_key['uCT_filename'] == filename, 'voltage'].iloc[0]
        min_voltage = sample_key.loc[sample_key['uCT_filename'] == filename, 'min_voltage'].iloc[0]
        max_voltage = sample_key.loc[sample_key['uCT_filename'] == filename, 'max_voltage'].iloc[0]
        df['Voltage Range'] = (((voltage-min_voltage)/(max_voltage-min_voltage))*100).round(0)
        
        #calculate maxiumum wall thickness
        df['fwall_mthickness'] = (df['fmajor_axis'] - df['lminor_axis'])/2
         
        #concatonate all of the lists into a single massive list
        cdf = pd.concat([df, cdf], axis=0,ignore_index=True,sort=False)
#print(cdf.head())
cdf.to_csv(save_loc+'MicroCT/porosity_data/processed/'+'cdf.csv')

## Processing fresh distributions to make summary file

In [ ]:
def removeoutliers(dataframe):
    Q1 = dataframe.quantile(0.25)
    Q3 = dataframe.quantile(0.75)
    IQR = Q3 - Q1
    new_dataframe = dataframe[~((dataframe < (Q1 - 1.5 * IQR)) |(dataframe > (Q3 + 1.5 * IQR))).any(axis=1)]
    return new_dataframe

def statskew(dataframe,property_column):
    property_median = dataframe[property_column].median()
    property_mean = dataframe[property_column].mean()
    property_skew = skew(dataframe[property_column])
    Q1 = dataframe[property_column].quantile(0.25)
    Q3 = dataframe[property_column].quantile(0.75)
    IQR = Q3 - Q1
    if abs(property_skew) > 0.5:
        property = property_median
    else:
        property = property_mean
    return property,IQR

## Unification of distribution summaries

In [ ]:
#creating dataframe for processed fibre properties
columns = ['fmajor_axis', 'fminor_axis', 'faspect_ratio', 'lmajor_axis', 'lminor_axis', 'laspect_ratio', 'porosity']
pfp = pd.DataFrame(columns = columns)

#First copy all raw data file which you want to compare into a single file and then insert directory of file below
#This then retreves the name of each file in the specified directory and cycle through them with the following
for filename in os.listdir(raw_pore+'preprocessed/'):
    #only open the file if it end with the specified parameters as specified in file name
    if filename.endswith(".csv"):      
        #reads the specified directory and opens it as a dataframe
        df = pd.read_csv(os.path.join(raw_pore+'preprocessed/',filename))
        
        pore_min = df['porosity'].min()
        pore_max = df['porosity'].max()
        
        #screening outliers from dataframe
        sfp = removeoutliers(df)

        #defining the statistical centre of each measured value - taking the statistical measure depending on skew
        fmajor_axis,fmajor_IQR = statskew(sfp,'fmajor_axis')
        fminor_axis,fminor_IQR = statskew(sfp,'fminor_axis')
        faspect_ratio,faspect_IQR = statskew(sfp,'faspect_ratio')
        lmajor_axis,lmajor_IQR = statskew(sfp,'lmajor_axis')
        lminor_axis,lminor_IQR = statskew(sfp,'lminor_axis')
        laspect_ratio,laspect_IQR = statskew(sfp,'laspect_ratio')
        porosity,pore_IQR = statskew(sfp,'porosity')
        fwall_mthickness = (fmajor_axis - lminor_axis)/2
        fwall_mthickness_IQR = (fmajor_IQR + lminor_IQR)/2

        #Import key fibre information
        Pyridine_Concentration = sample_key.loc[sample_key['uCT_filename'] == filename, 'pyridine_conc'].iloc[0]
        Wire_Speed = sample_key.loc[sample_key['uCT_filename'] == filename, 'wire_speed'].iloc[0]
        Rotation_Speed = sample_key.loc[sample_key['uCT_filename'] == filename, 'rotation_speed'].iloc[0]
        abs_pcl_conc = sample_key.loc[sample_key['uCT_filename'] == filename, 'abs_pcl_conc'].iloc[0]
        abs_pdo_conc = sample_key.loc[sample_key['uCT_filename'] == filename, 'abs_pdo_conc'].iloc[0]
        Polymer_Composition = ((abs_pcl_conc*100).round(2).astype(str) +'%PCL')+'\n'+((abs_pdo_conc*100).round(2).astype(str) +'%PDO')
        voltage = sample_key.loc[sample_key['uCT_filename'] == filename, 'voltage'].iloc[0]
        min_voltage = sample_key.loc[sample_key['uCT_filename'] == filename, 'min_voltage'].iloc[0]
        max_voltage = sample_key.loc[sample_key['uCT_filename'] == filename, 'max_voltage'].iloc[0]
        Voltage_Range = (((voltage-min_voltage)/(max_voltage-min_voltage))*100).round(0)
        
        #append processed information 
        pfp = pfp.append({'fmajor_axis':fmajor_axis, 'fminor_axis':fminor_axis, 'faspect_ratio':faspect_ratio, 
                          'lmajor_axis':lmajor_axis, 'lminor_axis':lminor_axis, 'laspect_ratio':laspect_ratio, 
                          'porosity':porosity, 'Pyridine Concentration':Pyridine_Concentration,'Wire Speed':Wire_Speed, 
                          'Rotation Speed':Rotation_Speed,'Polymer Composition':Polymer_Composition,
                          'Voltage Range': Voltage_Range,'pore_IQR':pore_IQR,'fmajor_IQR':fmajor_IQR,'fminor_IQR':fminor_IQR,
                          'faspect_IQR':faspect_IQR,'lmajor_IQR':lmajor_IQR,'lminor_IQR':lminor_IQR,'laspect_IQR':laspect_IQR,
                          'fwall_mthickness':fwall_mthickness,'fwall_mthickness_IQR':fwall_mthickness_IQR}, ignore_index=True)
        
       
        
        
pfp.to_csv(save_loc+'MicroCT/porosity_data/processed/'+'all_processed'+'.csv')
#print(pfp)

# Plotting

In [ ]:
import sys
print(sys.path)

In [ ]:
#Want to consider the effect of each variable however as data is not filtered need to filter to evaluate the effect of only a single variable at a given time
def varlayer(layer_num,layer_lst):
    for layer_num in layer_lst:
        #find all of the unique values of the 2nd controlled variable
        uniquevalues = np.unique(grp2[layer_num].values)
        for id in uniquevalues:
            #create a dataframe which contains only fixed values of controlled variable 3
            #create grp1 from variable in variable list
            for key, nxtgrp in grp.groupby([layer_num]):
                #To continue fixing the variable now shorten variable list to consider only other uncontrolled variables
                nxt_layer_lst = layer_lst.copy() #list of secondary controlled variables
                #removing x-axis variable so only consider changing variables
                nxt_layer_lst.remove(layer_num)
                return nxtgrp,nxt_layer_lst

## From preprocessed data

In [ ]:
#first define variables to be quiried 
variables = ['Rotation Speed','Wire Speed','Pyridine Concentration','Polymer Solution','Voltage Range']
#import summary data
df = pd.read_csv(os.path.join(save_loc+'/'+'MicroCT'+'/'+'porosity_data'+'/'+'processed'+'/'+'cdf.csv'),index_col=0)

#Filter summariesed data to isolate single variable effect
for variable in variables:
        #find all of the unique values of the 1st controlled variable
        uniquevalues = np.unique(df[variable].values)
        #for each of the different variables make dataframe further sub divided
        for id1 in uniquevalues:
            #create a dataframe which contains only fixed values of controlled variable 1
            newdf1 = df[df[variable] == id1]
            #To continue fixing the variable now shorten variable list to consider only other uncontrolled variables
            #starting from x_hue
            variable_lst2 = variables.copy() #list of secondary controlled variables
            #removing x-axis variable so only consider changing variables
            variable_lst2.remove(variable)
            
            for variable2 in variable_lst2:
                #find all of the unique values of the 1st controlled variable
                uniquevalues = np.unique(newdf1[variable2].values)
                #for each of the different variables make dataframe further sub divided
                for id2 in uniquevalues:
                    #create a dataframe which contains only fixed values of controlled variable 2
                    newdf2 = newdf1[newdf1[variable2] == id2]
                    #To continue fixing the variable now shorten variable list to consider only other uncontrolled variables
                    #starting from x_hue
                    variable_lst3 = variable_lst2.copy() #list of secondary controlled variables
                    #removing x-axis variable so only consider changing variables
                    variable_lst3.remove(variable2)
                    
                    for variable3 in variable_lst3:
                        #find all of the unique values of the 1st controlled variable
                        uniquevalues = np.unique(newdf2[variable3].values)
                        #for each of the different variables make dataframe further sub divided
                        for id3 in uniquevalues:
                            #create a dataframe which contains only fixed values of controlled variable 3
                            newdf3 = newdf2[newdf2[variable3] == id3]
                            #To continue fixing the variable now shorten variable list to consider only other uncontrolled variables
                            #starting from x_hue
                            variable_lst4 = variable_lst3.copy() #list of secondary controlled variables
                            #removing x-axis variable so only consider changing variables
                            variable_lst4.remove(variable3)
                            
                            for variable4 in variable_lst4:
                                #starting from x_hue
                                variable_lst4 = variable_lst3.copy() #list of secondary controlled variables
                                #removing x-axis variable so only consider changing variables
                                variable_lst4.remove(variable3)
                                #find all of the unique values of the 1st controlled variable
                                uniquevalues = np.unique(newdf3[variable4].values)
                                #for each of the different variables make dataframe further sub divided
                                
                                for id4 in uniquevalues:
                                    #starting from x_hue
                                    variable_lst5 = variable_lst4.copy() #list of secondary controlled variables
                                    #removing x-axis variable so only consider changing variables
                                    variable_lst5.remove(variable4)
                                    #create a dataframe which contains only fixed values of controlled variable 4
                                    newdf4 = newdf3[newdf3[variable4] == id4]
                                    
                                    #In order to prevent duplications of files with same fixed variables but in different order
                                    #make list of variables and order them
                                    fxd_ivars = [str(id1),str(id2),str(id3),str(id4)]
                                    fxd_ivars = sorted(fxd_ivars)
                                                                        
                                    #identify the different vaiables present to allow for potential statistical comparison
                                    uniquevalues = np.unique(newdf4[variable_lst5[0]].values)

                                    #create list of valuesvalues = []
                                    values = []
                                    #for each value remove PPM and convert to interger
                                    for value in uniquevalues:
                                        values.append(value)
                                    #sort the list of values according to size
                                    values.sort(reverse = False)
                                    
                                    
                                    #if there are more than one variable continue to plot
                                    if len(values) > 2:         

                                        #Within the uncontrolled variable want to identify all the condidtions used
                                        #uniquevalues = np.unique(newdf4[variable_lst5[0]].values)

                                        #For each combination of the controlled variables want to evaluate how each metric changes
                                        metrics = ['fmajor_axis','fminor_axis','faspect_ratio','lmajor_axis','lminor_axis',
                                                   'laspect_ratio','porosity','fwall_mthickness']
                                        #Evaluating the change in each of the metrics
                                        for metric in metrics:
                                            
                                           
                                            #for each oof the conditions used in the uncontrolled variable
                                            for id5 in uniquevalues:
                                                #To prevent duplication check if quiry has been made
                                                if not os.path.isfile(save_loc+'/'+'MicroCT/'+'figures'+'/'+str(variable_lst5[0])+str(fxd_ivars)+metric+'.png'):
                                                    #isolate the data associated for the same conditions used 
                                                    #newdf5 = newdf4[newdf4[variable_lst5[0]] == id5]
                                                    
                                                    
                                                    #To be able to screen outliers with this method cannot have strings in table as these cannot be compared with >
                                                    #For this reason have started to refer to different polymer solutions by numbers
                                                    #screening outliers from dataframe
                                                    #newdf4 = removeoutliers(newdf4,variable_lst5[0])
                                                    #print(newdf4)
                                                    Q1 = newdf4.quantile(0.25)
                                                    #print(Q1)
                                                    Q3 = newdf4.quantile(0.75)
                                                    #print(Q3)
                                                    IQR = Q3 - Q1
                                                    newdf4 = newdf4[~((newdf4 < (Q1 - 1.5 * IQR)) |(newdf4 > (Q3 + 1.5 * IQR))).any(axis=1)]
                                                    #print(newdf4)
                                                    #print(snewdf4)
                                                    
                                                    #Create new figure for each metric considered
                                                    fig, ax = plt.subplots()
                                                    
                                                    #make scatter plot of data associated with each of the metrics
                                                    #fig1 = ax.errorbar(newdf5[variable_lst5[0]], newdf5[metric], fmt='o',label=label)
                                                    fig = sns.violinplot(x= newdf4[variable_lst5[0]], y= newdf4[metric], showfliers=True, order= values,ax=ax)
                                                    #to account for solutions being refered to by number now must extract number from figure
                                                    if variable_lst5[0] == 'Polymer Solution':
                                                        polysolkey = {'0.0':'Trial','1.0':'Initial' ,'2.0':'S1','3.0':'S2',
                                                                      '4.0':'S3','5.0':'S4','6.0':'S5'}
                                                        labels = [t.get_text()  for t in ax.get_xticklabels()]
                                                    #    print(labels)
                                                        new_labels = [polysolkey[l] for l in labels]
                                                    #    print(new_labels)
                                                        fig.set_xticklabels(new_labels)

                                                    #before can add statistical annotation must create boxPairList from previous statistical comparison table
                                                    #create list for boxpairlist
                                                    pre_boxPairList = []


                                                    #for count of number of o values
                                                    for index in range(len(values)):
                                                        #to ensure that all combinations are considered again copy the uniquevalues
                                                        avalues = values.copy() #colour hue
                                                        #removing fixed variable so only consider changing variables
                                                        avalues.remove(values[index])
                                                        #considereing the appending value
                                                        for index in range(len(avalues)):
                                                            #let a = the file name and the ovalue which corresponds to the number within the list and pair them
                                                            a = (avalues[index],values[index])
                                                            #add the pair to the list of boxed pairs
                                                            if avalues[index] != values[index]:
                                                                if a not in pre_boxPairList:
                                                                    pre_boxPairList.append(a)
                                                            else: pass
                                                    #pre_boxPairList = sorted(pre_boxPairList)
                                                    #print(pre_boxPairList)

                                                    #add statistical annotations
                                                    add_stat_annotation(x= newdf4[variable_lst5[0]], y= newdf4[metric],order= values,showfliers=True,
                                                                        boxPairList=pre_boxPairList,test='Mann-Whitney', textFormat='star', loc='inside', verbose=0,
                                                                        ax=ax)

                                                    #label axis
                                                    #create dictionary of y-axis labels associated with each of the metrics
                                                    y_lables = {'fmajor_axis':'Maximum Fibre Diameter ($\mu$m)','fminor_axis':'Minimum Fibre Diameter ($\mu$m)',
                                                                'faspect_ratio':'Fibre Aspect Ratio','lmajor_axis':'Maximum Lumen Diameter ($\mu$m)',
                                                                'lminor_axis':'Minimum Lumen Diameter ($\mu$m)','laspect_ratio':'Lumen Aspect Ratio',
                                                                'porosity':'Porosity (%)','fwall_mthickness':'Maximum Fibre Wall thickness'}

                                                    x_lables = {'Polymer Composition':'Polymer Composition','Pyridine Concentration':'Pyridine Concentration (PPM)',
                                                               'Polymer Solution':'Polymer Solution'}

                                                    ax.set_ylabel(y_lables[metric])
                                                    ax.set_xlabel(x_lables[variable_lst5[0]])

                                                    #bx_data = [bx_q1,bx_med,bx_q3]

                                                    #ax = sns.boxplot(data=bx_data)

                                                    #after looping through all of the catagories, save figure
                                                    ax.figure.savefig(save_loc+'/'+'MicroCT/'+'figures'+'/'+str(variable_lst5[0])+str(fxd_ivars)+metric+'.png',bbox_inches='tight', dpi=300)

## From Summary Data

In [ ]:
#first define variables to be quiried 
variables = ['Rotation Speed','Wire Speed','Pyridine Concentration','Polymer Composition','Voltage Range']
#import summary data
df = pd.read_csv(os.path.join(save_loc+'/'+'MicroCT'+'/'+'porosity_data'+'/'+'processed'+'/'+'all_processed.csv'))
                   
#Filter summariesed data to isolate single variable effect
for variable in variables:
        #find all of the unique values of the 1st controlled variable
        uniquevalues = np.unique(df[variable].values)
        #for each of the different variables make dataframe further sub divided
        for id1 in uniquevalues:
            #create a dataframe which contains only fixed values of controlled variable 1
            newdf1 = df[df[variable] == id1]
            #To continue fixing the variable now shorten variable list to consider only other uncontrolled variables
            #starting from x_hue
            variable_lst2 = variables.copy() #list of secondary controlled variables
            #removing x-axis variable so only consider changing variables
            variable_lst2.remove(variable)
            
            for variable2 in variable_lst2:
                #find all of the unique values of the 1st controlled variable
                uniquevalues = np.unique(newdf1[variable2].values)
                #for each of the different variables make dataframe further sub divided
                for id2 in uniquevalues:
                    #create a dataframe which contains only fixed values of controlled variable 2
                    newdf2 = newdf1[newdf1[variable2] == id2]
                    #To continue fixing the variable now shorten variable list to consider only other uncontrolled variables
                    #starting from x_hue
                    variable_lst3 = variable_lst2.copy() #list of secondary controlled variables
                    #removing x-axis variable so only consider changing variables
                    variable_lst3.remove(variable2)
                    
                    for variable3 in variable_lst3:
                        #find all of the unique values of the 1st controlled variable
                        uniquevalues = np.unique(newdf2[variable3].values)
                        #for each of the different variables make dataframe further sub divided
                        for id3 in uniquevalues:
                            #create a dataframe which contains only fixed values of controlled variable 3
                            newdf3 = newdf2[newdf2[variable3] == id3]
                            #To continue fixing the variable now shorten variable list to consider only other uncontrolled variables
                            #starting from x_hue
                            variable_lst4 = variable_lst3.copy() #list of secondary controlled variables
                            #removing x-axis variable so only consider changing variables
                            variable_lst4.remove(variable3)
                            
                            for variable4 in variable_lst4:
                                #starting from x_hue
                                variable_lst4 = variable_lst3.copy() #list of secondary controlled variables
                                #removing x-axis variable so only consider changing variables
                                variable_lst4.remove(variable3)
                                #find all of the unique values of the 1st controlled variable
                                uniquevalues = np.unique(newdf3[variable4].values)
                                #for each of the different variables make dataframe further sub divided
                                
                                for id4 in uniquevalues:
                                    #starting from x_hue
                                    variable_lst5 = variable_lst4.copy() #list of secondary controlled variables
                                    #removing x-axis variable so only consider changing variables
                                    variable_lst5.remove(variable4)
                                    #create a dataframe which contains only fixed values of controlled variable 4
                                    newdf4 = newdf3[newdf3[variable4] == id4]
                                    
                                    #save the new dataframe as a csv - to be compared later on
                                    #In order to prevent duplications of files with same fixed variables but in different order
                                    #make list of variables and order them
                                    fxd_ivars = [str(id1),str(id2),str(id3),str(id4)]
                                    fxd_ivars = sorted(fxd_ivars)
                                    
                                    #To prevent duplication check if quiry has been made
                                    if not os.path.isfile(save_loc+'/'+'MicroCT/'+'figures'+'/'+str(variable_lst5[0])+str(fxd_ivars)+metric+'.png'):
                                        
                                        #identify the different vaiables present to allow for potential statistical comparison
                                        uniquevalues = np.unique(newdf4[variable_lst5[0]].values)
                                        
                                        #create list of values
                                        values = []
                                        #for each value remove PPM and convert to interger
                                        for value in uniquevalues:
                                            values.append(value)
                                        #sort the list of values according to size
                                        values.sort(reverse = True)

                                        #before can add statistical annotation must create boxPairList from previous statistical comparison table
                                        #create list for boxpairlist
                                        pre_boxPairList = []


                                        #for count of number of o values
                                        for index in range(len(values)):
                                            #to ensure that all combinations are considered again copy the uniquevalues
                                            avalues = values.copy() #colour hue
                                            #removing fixed variable so only consider changing variables
                                            avalues.remove(values[index])
                                            #considereing the appending value
                                            for index in range(len(avalues)):
                                                #let a = the file name and the ovalue which corresponds to the number within the list and pair them
                                                a = (avalues[index],values[index])
                                                #add the pair to the list of boxed pairs
                                                if avalues[index] != values[index]:
                                                    if a not in pre_boxPairList:
                                                        pre_boxPairList.append(a)
                                                else: pass
                                        #pre_boxPairList = sorted(pre_boxPairList)
                                        #print(pre_boxPairList)
                                        
                                        #if there are more than one variable continue to plot
                                        if len(pre_boxPairList) > 1:
                                            
                                            #Within the uncontrolled variable want to identify all the condidtions used
                                            uniquevalues = np.unique(newdf4[variable_lst5[0]].values)
                                            
                                            #For each combination of the controlled variables want to evaluate how each metric changes
                                            metrics = ['fmajor_axis','fminor_axis','faspect_ratio','lmajor_axis','lminor_axis',
                                                       'laspect_ratio','porosity','fwall_mthickness']
                                            #Evaluating the change in each of the metrics
                                            for metric in metrics:
                                                #Create new figure for each metric considered
                                                fig, ax = plt.subplots()
                                                #for each oof the conditions used in the uncontrolled variable
                                                for id5 in uniquevalues:
                                                    #isolate the data associated for the same conditions used 
                                                    newdf5 = newdf4[newdf4[variable_lst5[0]] == id5]
                                                    
                                                    #create dictionary of IQRs associated with each of the metrics
                                                    IQRs = {'fmajor_axis':'fmajor_IQR','fminor_axis':'fminor_IQR','faspect_ratio':'faspect_IQR',
                                                            'lmajor_axis':'lmajor_IQR','lminor_axis':'lminor_IQR','laspect_ratio':'laspect_IQR',
                                                            'porosity':'pore_IQR','fwall_mthickness':'fwall_mthickness_IQR'}
                                                    
                                                    #make scatter plot of data associated with each of the metrics
                                                    fig1 = ax.errorbar(newdf5[variable_lst5[0]], newdf5[metric], yerr=newdf5[IQRs[metric]], fmt='o',label=label)
                                                    
                                                    #label axis
                                                    #create dictionary of y-axis labels associated with each of the metrics
                                                    y_lables = {'fmajor_axis':'Maximum Fibre Diameter ($\mu$m)','fminor_axis':'Minimum Fibre Diameter ($\mu$m)',
                                                                'faspect_ratio':'Fibre Aspect Ratio','lmajor_axis':'Maximum Lumen Diameter ($\mu$m)',
                                                                'lminor_axis':'Minimum Lumen Diameter ($\mu$m)','laspect_ratio':'Lumen Aspect Ratio',
                                                                'porosity':'Porosity (%)','fwall_mthickness':'Maximum Fibre Wall thickness'}
                                                    
                                                    x_lables = {'Polymer Composition':'Polymer Composition','Pyridine Concentration':'Pyridine Concentration (PPM)'}

                                                    ax.set_ylabel(y_lables[metric])
                                                    ax.set_xlabel(x_lables[variable_lst5[0]])

                                                    bx_data = [bx_q1,bx_med,bx_q3]

                                                    #ax = sns.boxplot(data=bx_data)

                                                #after looping through all of the catagories, save figure
                                                ax.figure.savefig(save_loc+'/'+'MicroCT/'+'figures'+'/'+str(variable_lst5[0])+str(fxd_ivars)+metric+'.png',bbox_inches='tight', dpi=300)

  